In [ ]:
# 1. Funktionen und Bibliotheken

# Löscht alle Variablen aus der aktuellen Sitzung um sicherzustellen 
# dass keine alten Werte oder Definitionen den Code beeinflussen.
%reset -f

# Importieren der benötigten Bibliotheken
import numpy as np  # Für numerische Berechnungen
import pandas as pd  # Für die Arbeit mit Tabellen Datensätzen
import matplotlib.pyplot as plt  # Für graphische Darstellung
from sklearn.model_selection import train_test_split  # Für das Teilen der Daten in Trainings- und Testsets
from sklearn.linear_model import LinearRegression  # Für die lineare Regressionsanalyse
from sklearn.metrics import mean_squared_error  # Für die Berechnung der mittleren quadratischen Abweichung (MSE)

# Definition einer Funktion für die Berechnung von Punkten auf einer quartischen Bézierkurve
def quartic_bezier(points, t):
    # Berechnet einen Punkt auf der Bézierkurve für den gegebenen Parameter t
    return ((1 - t)**4 * points[0] +              # Einfluss des ersten Kontrollpunkts
            4 * (1 - t)**3 * t * points[1] +     # Einfluss des zweiten Kontrollpunkts
            6 * (1 - t)**2 * t**2 * points[2] +  # Einfluss des dritten Kontrollpunkts
            4 * (1 - t) * t**3 * points[3] +     # Einfluss des vierten Kontrollpunkts
            t**4 * points[4])                   # Einfluss des fünften Kontrollpunkts

# Funktion zur Generierung von 100 Punkten auf einer Bézierkurve
def generate_bezier_points(points, num_points=100):
    # Generiert num_points Punkte auf der Bézierkurve
    return np.array([quartic_bezier(points, t) for t in np.linspace(0, 1, num_points)])

# Funktion zur Berechnung der Länge einer Kurve
def curve_length(curve):
    # Berechnet die Gesamtlänge einer Kurve basierend auf den Distanzen zwischen aufeinanderfolgenden Punkten
    distances = np.sqrt(np.sum(np.diff(curve, axis=0)**2, axis=1))
    return np.sum(distances)

# Funktion zur Berechnung der Distanz zwischen zwei Punkten
def point_distance(p1, p2):
    p1 = np.array(p1)  # Umwandeln der Punkte in numpy Arrays
    p2 = np.array(p2)
    # Berechnung der euklidischen Distanz
    return np.sqrt(np.sum((p1 - p2)**2))

In [ ]:
# 2. Bézierkurven zufallsgenerieren und vermessen
# Dies erzeugt die Daten für späteres Maschinenlernen
# n kann angepasst werden, aber bei großem n dauert es lange zu rendern

# Anzahl der Bézierkurven, die erzeugt werden sollen
n = 1000 

# Generierung von zufälligen Kontrollpunkten für n Bézierkurven
# Jede Bézierkurve hat 5 Kontrollpunkte im 2D-Raum
punkte_quartic = np.random.randint(low=0, high=30, size=(n, 5, 2))

# Initialisierung einer Liste zum Speichern der Ergebnisse
data = []

# Schleife über jede Gruppe von Kontrollpunkten
for i in range(n):
    control_points = punkte_quartic[i]  # Die Kontrollpunkte der aktuellen Bézierkurve
    bezier_points = generate_bezier_points(control_points)  # Generierung der Bézierkurve
    length = curve_length(bezier_points)  # Berechnung der Länge der Bézierkurve
    
    # Berechnung der Distanzen zwischen den Kontrollpunkten
    d01 = point_distance(control_points[0], control_points[1])  # Distanz zwischen P0 und P1
    d02 = point_distance(control_points[0], control_points[2])  # Distanz zwischen P0 und P2
    d03 = point_distance(control_points[0], control_points[3])  # Distanz zwischen P0 und P3
    d04 = point_distance(control_points[0], control_points[4])  # Distanz zwischen P0 und P4
    d12 = point_distance(control_points[1], control_points[2])  # Distanz zwischen P1 und P2
    d13 = point_distance(control_points[1], control_points[3])  # Distanz zwischen P1 und P3
    d14 = point_distance(control_points[1], control_points[4])  # Distanz zwischen P1 und P4
    d23 = point_distance(control_points[2], control_points[3])  # Distanz zwischen P2 und P3
    d24 = point_distance(control_points[2], control_points[4])  # Distanz zwischen P2 und P4
    d34 = point_distance(control_points[3], control_points[4])  # Distanz zwischen P3 und P4
    
    # Speichern der Ergebnisse als Eintrag in der Tabelle
    entry = {
        'P0': control_points[0].tolist(),  # Koordinaten von P0
        'P1': control_points[1].tolist(),  # Koordinaten von P1
        'P2': control_points[2].tolist(),  # Koordinaten von P2
        'P3': control_points[3].tolist(),  # Koordinaten von P3
        'P4': control_points[4].tolist(),  # Koordinaten von P4
        'Length': length,  # Länge der Bézierkurve
        'D_P0_P1': d01,  # Distanz zwischen P0 und P1
        'D_P0_P2': d02,  # Distanz zwischen P0 und P2
        'D_P0_P3': d03,  # Distanz zwischen P0 und P3
        'D_P0_P4': d04,  # Distanz zwischen P0 und P4
        'D_P1_P2': d12,  # Distanz zwischen P1 und P2
        'D_P1_P3': d13,  # Distanz zwischen P1 und P3
        'D_P1_P4': d14,  # Distanz zwischen P1 und P4
        'D_P2_P3': d23,  # Distanz zwischen P2 und P3
        'D_P2_P4': d24,  # Distanz zwischen P2 und P4
        'D_P3_P4': d34,  # Distanz zwischen P3 und P4
    }
    data.append(entry)  # Hinzufügen des Eintrags

# Erstellen eines Pandas DataFrame aus der Ergebnissammlung
df_trainingsdaten = pd.DataFrame(data)

print("Die Kontrollpunkte der Kurven, Länge der Kurven, und Abstände zwischen den Kontrollpunkten:\n")
print(df_trainingsdaten)

In [ ]:
# 3. Plotten der ersten 15 zufallsgenerierten Bézierkurven zur Ansicht

fig, axes = plt.subplots(3, 5, figsize=(20, 12))  # 3x5 Raster für 15 Kurven
axes = axes.flatten()  # Umwandeln der Achsen in Eindimensionalität für einfachere Iteration

# Für die ersten 15 Bézierkurven
for idx, control_points in enumerate(punkte_quartic[:15]):  # Nur die ersten 15 Kontrollpunkt-Sets
    bezier_curve = generate_bezier_points(control_points)  # Punkte auf der Bézierkurve generieren
    
    ax = axes[idx]
    control_points = np.array(control_points)  # Sicherstellen, dass Kontrollpunkte ein NumPy-Array sind
    
    # Plot der Bézierkurve
    ax.plot(bezier_curve[:, 0], bezier_curve[:, 1], color='blue')  # Blaue Bézierkurve
    # Kontrollpunkte plotten
    ax.plot(control_points[:, 0], control_points[:, 1], 'ro--', markersize=5)  # Rote Kontrollpunkte
    ax.set_title(f"Kurve {idx + 1}", fontsize=15)  
    ax.tick_params(axis='both', labelsize=15)  # Schriftgröße für die Achsenticks

# Entfernen überschüssiger Subplots, falls weniger als 15 Kurven vorhanden
for ax in axes[len(punkte_quartic[:15]):]:
    ax.axis('off')

# Globale Legende hinzufügen
fig.legend(
    ['Bézierkurve', 'Kontrollpunkte'], 
    loc='lower center', 
    ncol=2, 
    fontsize=20, 
    frameon=False
)

# Layout anpassen und anzeigen
plt.tight_layout(rect=[0, 0.05, 1, 1])  # Platz für die globale Legende unten schaffen
plt.show()

In [ ]:
# 4. Modell fitten und ausgeben
# mit einigen Zusatzinformationen zur besseren Beurteilung

# Bestimmen und Ausgabe der kürzesten und längsten Länge, Durchschnitt und Median
min_length = df_trainingsdaten['Length'].round(5).min()        # Kürzeste Länge
max_length = df_trainingsdaten['Length'].round(5).max()        # Längste Länge
mean_length = df_trainingsdaten['Length'].mean().round(5)      # Durchschnitt
median_length = df_trainingsdaten['Length'].round(5).median()  # Median

print(
    f"Längen der Kurven: [{min_length}, {max_length}]\n"
    f"Mittelwert: {mean_length}\n"
    f"Median: {median_length}\n"
)

# Definition der Merkmale (Features) und des Zielwerts (Target)
# X: Eingabevariablen (Abstände zwischen den Kontrollpunkten)
X = df_trainingsdaten[[
    'D_P0_P1', 'D_P0_P2', 'D_P0_P3', 'D_P0_P4', 
    'D_P1_P2', 'D_P1_P3', 'D_P1_P4', 
    'D_P2_P3', 'D_P2_P4', 
    'D_P3_P4'
]]
# y: Zielvariable (Länge der Bézierkurve)
y = df_trainingsdaten['Length']

# Aufteilen der Daten in Trainings- und Testdatensätze
# 80% der Daten werden für das Training verwendet, 20% für das Testen
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Erstellen und Trainieren des Modells
# Initialisierung eines linearen Regressionsmodells
model = LinearRegression()
# Training des Modells mit den Trainingsdaten
model.fit(X_train, y_train)

# Vorhersagen mit dem Modell
# Vorhersage der Zielwerte basierend auf den Testdaten
y_pred = model.predict(X_test)

# Evaluierung des Modells
# Berechnung der mittleren quadratischen Abweichung (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")  # Ausgabe der Abweichung

# Ausgabe der Modellparameter
# Die Koeffizienten der linearen Regression zeigen die Gewichtungen der einzelnen Merkmale
coefficients = model.coef_
intercept = model.intercept_  # Modell-Intercept 
print(f"Intercept: {intercept}\n")  # Ausgabe des Intercept
features = X.columns  # Speichert die Spaltennamen von X in der Variablen features

# Labels für den Index erstellen, angepasst für die neuen Variablen
labels = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']

# Erstellen eines DataFrame und setzen der Labels als Index
coef_df = pd.DataFrame({
    'Variable': labels,
    'Gewicht': coefficients.round(5),
    'Feature': features
})

# Ausgabe der Modellkoeffizienten
print("Gewichte der Linearen Regression:")
print(coef_df)


In [ ]:
# 5. Plotten der realen Werte vs. der Vorhersage

# Anlage des Diagramms
plt.figure(figsize=(10, 6)) 
plt.scatter(y_test, y_pred, alpha=0.6, color='blue', label='Datenpunkte')  # Streudiagramm

# Hinzufügen der Diagonalen für die Vorhersage
min_value = min(y_test.min(), y_pred.min(), 0)  # Kleinster Wert in den Daten
max_value = max(y_test.max(), y_pred.max())  # Größter Wert in den Daten
plt.plot([min_value, max_value], [min_value, max_value], color='red', linestyle='--', label='Vorhersage')

# Achsenbeschriftungen und Titel
plt.xlabel('Tatsächliche Werte (y_test)', fontsize=12)
plt.ylabel('Vorhergesagte Werte (y_pred)', fontsize=12)
plt.title('Berechnete vs. vorhergesagte Werte', fontsize=14)

# Hinzufügen einer Legende und eines Gitters
plt.legend()
plt.grid(True)

# Plot anzeigen
plt.show()

In [ ]:
# 6. Einführen und Plotten von Testdaten aus realen Projekten
# Siehe https://github.com/benjaminwand/cookie-cutters

punkte_testdaten = [ 
    # cactus
    [[40.5, 31], [40.5, 5], [40.5, 0], [39, 0], [28, 0]],
    [[30, 90], [37, 90], [46, 77], [35, 54.5], [41, 54.5]],
    [[41, 54.5], [48, 54.5], [43, 70], [43, 81.5], [51, 81.5]],
    # chaosknoten
    [[37.8, 61.5], [36, 50], [45, 55], [68, 60], [70.5, 47]],
    [[37.5, 39.4], [36.5, 49], [40, 45], [50, 46], [69, 46]]
]

# Plotten der Bézierkurven aus den punkte_testdaten
fig, axes = plt.subplots(1, 5, figsize=(25, 5))  # 1x5 Raster für 5 Kurven
axes = axes.flatten()  # Achsen in flache Liste umwandeln

# Plot jeder Bézierkurve
for idx, control_points in enumerate(punkte_testdaten[:5]):  # Begrenzung auf 5 Plots
    bezier_curve = generate_bezier_points(np.array(control_points))  # Bézierkurve berechnen
    control_points = np.array(control_points)  # Kontrollpunkte in NumPy-Array umwandeln
    
    ax = axes[idx]
    # Bézierkurve plotten
    ax.plot(bezier_curve[:, 0], bezier_curve[:, 1], color="blue")
    # Kontrollpunkte plotten
    ax.plot(control_points[:, 0], control_points[:, 1], 'ro--', markersize=5)
    # Titel für jede Kurve
    ax.set_title(f"Bézierkurve {idx + 1}", fontsize=15)
    ax.tick_params(axis='both', labelsize=12)  # Schriftgröße für die Achsenticks

# Layout anpassen und anzeigen
plt.tight_layout()
plt.show()

In [ ]:
# 7. Berechnung und Ausgabe der Abstände zwischen den Kontrollpunkten für den Testdatensatz

test_daten = []
for control_points in punkte_testdaten:

    # Kontrollpunkte in NumPy-Array konvertieren
    control_points = np.array(control_points)

    bezier_points = generate_bezier_points(control_points)  # Generierung der Bézierkurve
    length = curve_length(bezier_points)  # Berechnung der Länge der Bézierkurve
    
    # Berechnung der Abstände zwischen den Kontrollpunkten
    d01 = point_distance(control_points[0], control_points[1])  # Distanz zwischen P0 und P1
    d02 = point_distance(control_points[0], control_points[2])  # Distanz zwischen P0 und P2
    d03 = point_distance(control_points[0], control_points[3])  # Distanz zwischen P0 und P3
    d04 = point_distance(control_points[0], control_points[4])  # Distanz zwischen P0 und P4
    d12 = point_distance(control_points[1], control_points[2])  # Distanz zwischen P1 und P2
    d13 = point_distance(control_points[1], control_points[3])  # Distanz zwischen P1 und P3
    d14 = point_distance(control_points[1], control_points[4])  # Distanz zwischen P1 und P4
    d23 = point_distance(control_points[2], control_points[3])  # Distanz zwischen P2 und P3
    d24 = point_distance(control_points[2], control_points[4])  # Distanz zwischen P2 und P4
    d34 = point_distance(control_points[3], control_points[4])  # Distanz zwischen P3 und P4
    
    # Speichern der Ergebnisse als Eintrag
    entry = {
        'D_P0_P1': d01,   # Distanz zwischen P0 und P1
        'D_P0_P2': d02,   # Distanz zwischen P0 und P2
        'D_P0_P3': d03,   # Distanz zwischen P0 und P3
        'D_P0_P4': d04,   # Distanz zwischen P0 und P4
        'D_P1_P2': d12,   # Distanz zwischen P1 und P2
        'D_P1_P3': d13,   # Distanz zwischen P1 und P3
        'D_P1_P4': d14,   # Distanz zwischen P1 und P4
        'D_P2_P3': d23,   # Distanz zwischen P2 und P3
        'D_P2_P4': d24,   # Distanz zwischen P2 und P4
        'D_P3_P4': d34,   # Distanz zwischen P3 und P4
        'Length': length, # Länge der Bézierkurve
    }
    test_daten.append(entry)

# Umwandeln der Testdaten in ein DataFrame
df_test = pd.DataFrame(test_daten)

# Definition der Features für fünf Kontrollpunkte
features = [
    'D_P0_P1', 'D_P0_P2', 'D_P0_P3', 'D_P0_P4', 
    'D_P1_P2', 'D_P1_P3', 'D_P1_P4', 
    'D_P2_P3', 'D_P2_P4', 
    'D_P3_P4'
]

# Berechnung und Hinzufügen der Vorhersagen für jede Zeile im DataFrame df_test
df_test['Predicted_Length'] = (
    df_test[features].dot(coefficients) + intercept
)

# Ausgabe des DataFrame
print("Die Kontrollpunkte der Kurven, Länge der Kurven, und Abstände zwischen den Kontrollpunkten:\n")
print(df_test)


In [ ]:
# 8. Ausgabe der mittleren quadratischen Abweichung 
# mit einigen Zusatzinformationen zur besseren Beurteilung

# Bestimmen und Ausgabe der kürzesten und längsten Länge
min_length = df_test['Length'].round(4).min()    # Kürzeste Länge
max_length = df_test['Length'].round(4).max()    # Längste Länge
mean_length = df_test['Length'].mean().round(4)  # Durchschnitt
median_length = df_test['Length'].round(4).median()  # Median

print(
    f"Längen der Kurven: [{min_length}, {max_length}]\n"
    f"Mittelwert: {mean_length}\n"
    f"Median: {median_length}\n"
)

# Vergleich der vorhergesagten und tatsächlichen Längen
mse = mean_squared_error(df_test['Length'], df_test['Predicted_Length'])
print(f"Mean Squared Error (MSE): {mse}")

In [ ]:
# 9. Erstellen des Plots der Vorhersage und Testdaten gegenüberstellt

plt.figure(figsize=(10, 6))
plt.scatter(df_test['Length'], df_test['Predicted_Length'], color='blue', alpha=0.6, label='Berechnete Werte')

# Hinzufügen der Diagonalen für perfekte Vorhersage
max_value = max(df_test['Length'].max(), df_test['Predicted_Length'].max())
min_value = min(df_test['Length'].min(), df_test['Predicted_Length'].min())
plt.plot([min_value, max_value], [min_value, max_value], color='red', linestyle='--', label='Vorhersage')

# Achsenbeschriftungen und Titel
plt.xlabel('Tatsächliche Länge (Length)', fontsize=12)
plt.ylabel('Vorhergesagte Länge (Predicted Length)', fontsize=12)
plt.title('Vergleich: Tatsächliche vs. vorhergesagte Länge', fontsize=14)
plt.legend()
plt.grid(True)

# Plot anzeigen
plt.show()